In [ ]:
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix
import time
from threading import Thread
from threading import Event
from GammaClasses import PE
from GammaClasses import Scheduler
from GammaClasses import FiberCache
from GammaClasses import Memory

In [ ]:
I = 1000
K = 1000
J = 1000
NUM_INTS = 1000
NUM_PES = 32
RADIX = 64

gen = np.random.default_rng()
data1 = gen.integers(1,10,NUM_INTS)
row1 = gen.integers(0,I,NUM_INTS)
col1 = gen.integers(0,K,NUM_INTS)

data2 = gen.integers(1,10,NUM_INTS)
row2 = gen.integers(0,K,NUM_INTS)
col2 = gen.integers(0,J,NUM_INTS)
i1 = csr_matrix(coo_matrix((data1, (row1, col1)), shape=(I, K)))
i2 = csr_matrix(coo_matrix((data2, (row2, col2)), shape=(K, J)))
print(i1.toarray())
print(i2.toarray())

In [ ]:
FC = FiberCache(3000000,i2)
M = Memory(16,8)
PEArray = []
for x in range(NUM_PES):
    PEArray.append(PE(RADIX))

S = Scheduler(PEArray,i1,RADIX)

for x in S.jobs:
    print(x)

FC.setMemory(M)
for x in range(NUM_PES):
    PEArray[x].connect(S,FC)


In [ ]:
endFlag = False
while not endFlag:
    endFlag = True
    for x in range(NUM_PES):
        endFlag = endFlag and PEArray[x].endFlag
        PEArray[x].cycle()
    FC.cycle()
    M.cycle()

In [ ]:
#for x in range(NUM_PES):
#    
#    if not PEArray[x].endFlag:
#        print(PEArray[x].currentFiberSet)
#        print(PEArray[x].dataNotInCacheFlag)
#print(M.channelCurrent)
#print(FC.partialFiberLockSet)
#print(FC.remainingSize)
#print(FC.fetchSet)
#print(FC.fetchQueue)

In [ ]:
print(FC.outputMatrixCoords)
print(FC.outputMatrixData)
print(FC.outputMatrixIndptrs)
i = []
point = 0
for x in FC.outputMatrixIndptrs:
    i += [x[0]] * x[1]
print(i)

out = csr_matrix((FC.outputMatrixData, (i, FC.outputMatrixCoords)),shape=(I,J))

In [ ]:
print(np.matmul(i1.toarray(),i2.toarray()))
print(out.toarray())
print(np.equal(out.toarray(), np.matmul(i1.toarray(),i2.toarray())))
print(np.allclose(out.toarray(),np.matmul(i1.toarray(),i2.toarray()),rtol=0.01))
print(np.sum(out.toarray()))
print(np.sum(np.matmul(i1.toarray(),i2.toarray())))